<a href="https://colab.research.google.com/github/chettkulkarni/Reinforcement_Learning/blob/main/RL_Dl_homework_Chetan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL Homework

In this exercise we are running MSIT classifier using basic convnet.

And then we will be implememnting nAdam optimser and use he initialisation technique to initialise weights of the model

# Importing libraries

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
torch.manual_seed(0)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Intialisation of basic hyoer parameters

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# importing dataset

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Basic ConvNet for classification

In [5]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out



# Train All function to  initiate model training and testing multiple times

In [6]:
def train_all():
  total_step = len(train_loader)
  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):
          images = images.to(device)
          labels = labels.to(device)
          
          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)
          
          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          if (i+1) % 100 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                    .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
                    
  # Test the model

  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in test_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Running the model with Adam Optimiser and loss function of cross entropy

In [7]:
model = ConvNet(num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [8]:
train_all()

Epoch [1/5], Step [100/600], Loss: 0.1006
Epoch [1/5], Step [200/600], Loss: 0.1255
Epoch [1/5], Step [300/600], Loss: 0.1205
Epoch [1/5], Step [400/600], Loss: 0.0949
Epoch [1/5], Step [500/600], Loss: 0.1303
Epoch [1/5], Step [600/600], Loss: 0.0349
Epoch [2/5], Step [100/600], Loss: 0.0436
Epoch [2/5], Step [200/600], Loss: 0.0510
Epoch [2/5], Step [300/600], Loss: 0.0259
Epoch [2/5], Step [400/600], Loss: 0.0178
Epoch [2/5], Step [500/600], Loss: 0.0552
Epoch [2/5], Step [600/600], Loss: 0.0610
Epoch [3/5], Step [100/600], Loss: 0.0700
Epoch [3/5], Step [200/600], Loss: 0.0471
Epoch [3/5], Step [300/600], Loss: 0.0079
Epoch [3/5], Step [400/600], Loss: 0.0281
Epoch [3/5], Step [500/600], Loss: 0.0054
Epoch [3/5], Step [600/600], Loss: 0.0514
Epoch [4/5], Step [100/600], Loss: 0.0948
Epoch [4/5], Step [200/600], Loss: 0.0256
Epoch [4/5], Step [300/600], Loss: 0.0575
Epoch [4/5], Step [400/600], Loss: 0.0074
Epoch [4/5], Step [500/600], Loss: 0.0347
Epoch [4/5], Step [600/600], Loss:

### Accuracy of 99.06% with adam classifier and no initialisation

# He Intialisation

Here we will be intialisae weights using He intialisation technique.

Training the model with initilised weights and Adam optimser and cross entropy loss

In [10]:
def initialise_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_uniform_(m.weight.data)
        nn.init.zeros_(m.bias.data)

model = ConvNet(num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()       
model.apply(initialise_weights)
train_all()

Epoch [1/5], Step [100/600], Loss: 0.3438
Epoch [1/5], Step [200/600], Loss: 0.1632
Epoch [1/5], Step [300/600], Loss: 0.0707
Epoch [1/5], Step [400/600], Loss: 0.1313
Epoch [1/5], Step [500/600], Loss: 0.0242
Epoch [1/5], Step [600/600], Loss: 0.0677
Epoch [2/5], Step [100/600], Loss: 0.0154
Epoch [2/5], Step [200/600], Loss: 0.0286
Epoch [2/5], Step [300/600], Loss: 0.0417
Epoch [2/5], Step [400/600], Loss: 0.0346
Epoch [2/5], Step [500/600], Loss: 0.0129
Epoch [2/5], Step [600/600], Loss: 0.1204
Epoch [3/5], Step [100/600], Loss: 0.0172
Epoch [3/5], Step [200/600], Loss: 0.0298
Epoch [3/5], Step [300/600], Loss: 0.0306
Epoch [3/5], Step [400/600], Loss: 0.0182
Epoch [3/5], Step [500/600], Loss: 0.0145
Epoch [3/5], Step [600/600], Loss: 0.0224
Epoch [4/5], Step [100/600], Loss: 0.0386
Epoch [4/5], Step [200/600], Loss: 0.0203
Epoch [4/5], Step [300/600], Loss: 0.0414
Epoch [4/5], Step [400/600], Loss: 0.0193
Epoch [4/5], Step [500/600], Loss: 0.0403
Epoch [4/5], Step [600/600], Loss:

### With the initiased weights the accuracy of system fell down to 98.8%

# Trying with NAdam Optimser

its not available directly, but there is a distribution by torch on github for the same

[GitHub Pull for Nadam optimiser](https://github.com/pytorch/pytorch/pull/1414/files)

In [11]:
import math
from torch.optim.optimizer import Optimizer

class Nadam(Optimizer):
    """Implements Nadam algorithm.
    It has been proposed in `Incorporating Nesterov Momentum into Adam`_.
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 2e-3)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.975, 0.999))
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        schedule_decay (float, optional): beta1 decay factor (default: 0)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
    .. _Incorporating Nesterov Momentum into Adam
        https://openreview.net/pdf?id=OM0jvwB8jIp57ZJjtNEZ
    """

    def __init__(self, params, lr=2e-3, betas=(0.975, 0.999), eps=1e-8,
                 schedule_decay=0, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        schedule_decay=schedule_decay, weight_decay=weight_decay,
                        prod_beta1=1.)
        super(Nadam, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            beta1, beta2 = group['betas']

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = grad.new().resize_as_(grad).zero_()
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = grad.new().resize_as_(grad).zero_()

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                state['step'] += 1

                if group['weight_decay'] != 0:
                    grad = grad.add(group['weight_decay'], p.data)

                schedule_decay = group['schedule_decay']
                cur_beta1 = beta1 * (1. - 0.5 * (0.96 ** (state['step'] * schedule_decay)))
                next_beta1 = beta1 * (1. - 0.5 * (0.96 ** ((state['step'] + 1) * schedule_decay)))
                prod_beta1 = group['prod_beta1']
                prod_beta1 *= cur_beta1
                next_prod_beta1 = prod_beta1 * next_beta1
                bias_correction1 = (1 - cur_beta1) / (1 - prod_beta1)
                next_bias_correction1 = next_beta1 / (1 - next_prod_beta1)

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(cur_beta1).add_(1 - cur_beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                sqrt_bias_correction2 = math.sqrt((1 - beta2 ** state['step']) / beta2)
                step_size = group['lr'] * sqrt_bias_correction2

                denom = exp_avg_sq.sqrt().add_(group['eps'])

                # For memory efficiency, separate update into two
                p.data.addcdiv_(-step_size * next_bias_correction1, exp_avg, denom)
                p.data.addcdiv_(-step_size * bias_correction1, grad, denom)

                # update prod_beta1
                group['prod_beta1'] = prod_beta1

        return loss

In [12]:
model = ConvNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Nadam(model.parameters(), lr=learning_rate)
train_all()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)


Epoch [1/5], Step [100/600], Loss: 0.1679
Epoch [1/5], Step [200/600], Loss: 0.1061
Epoch [1/5], Step [300/600], Loss: 0.0930
Epoch [1/5], Step [400/600], Loss: 0.1010
Epoch [1/5], Step [500/600], Loss: 0.0846
Epoch [1/5], Step [600/600], Loss: 0.0186
Epoch [2/5], Step [100/600], Loss: 0.0587
Epoch [2/5], Step [200/600], Loss: 0.0406
Epoch [2/5], Step [300/600], Loss: 0.0251
Epoch [2/5], Step [400/600], Loss: 0.0284
Epoch [2/5], Step [500/600], Loss: 0.0061
Epoch [2/5], Step [600/600], Loss: 0.0487
Epoch [3/5], Step [100/600], Loss: 0.0134
Epoch [3/5], Step [200/600], Loss: 0.0031
Epoch [3/5], Step [300/600], Loss: 0.0335
Epoch [3/5], Step [400/600], Loss: 0.0417
Epoch [3/5], Step [500/600], Loss: 0.0159
Epoch [3/5], Step [600/600], Loss: 0.0287
Epoch [4/5], Step [100/600], Loss: 0.0030
Epoch [4/5], Step [200/600], Loss: 0.0190
Epoch [4/5], Step [300/600], Loss: 0.0072
Epoch [4/5], Step [400/600], Loss: 0.0162
Epoch [4/5], Step [500/600], Loss: 0.0101
Epoch [4/5], Step [600/600], Loss:

### When we changes the optiser to nAdam , accuracy went to 98.79 , which is lower than the original when we had adam optimiser

# Using Both Nadam and "he" weight intitialsation 

In [13]:
model = ConvNet(num_classes).to(device)
optimizer = Nadam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()       
model.apply(initialise_weights)
train_all()

Epoch [1/5], Step [100/600], Loss: 0.2094
Epoch [1/5], Step [200/600], Loss: 0.0958
Epoch [1/5], Step [300/600], Loss: 0.1001
Epoch [1/5], Step [400/600], Loss: 0.1273
Epoch [1/5], Step [500/600], Loss: 0.0613
Epoch [1/5], Step [600/600], Loss: 0.0670
Epoch [2/5], Step [100/600], Loss: 0.0207
Epoch [2/5], Step [200/600], Loss: 0.0361
Epoch [2/5], Step [300/600], Loss: 0.0618
Epoch [2/5], Step [400/600], Loss: 0.0824
Epoch [2/5], Step [500/600], Loss: 0.0865
Epoch [2/5], Step [600/600], Loss: 0.0433
Epoch [3/5], Step [100/600], Loss: 0.0800
Epoch [3/5], Step [200/600], Loss: 0.0495
Epoch [3/5], Step [300/600], Loss: 0.0686
Epoch [3/5], Step [400/600], Loss: 0.0248
Epoch [3/5], Step [500/600], Loss: 0.0715
Epoch [3/5], Step [600/600], Loss: 0.0511
Epoch [4/5], Step [100/600], Loss: 0.0104
Epoch [4/5], Step [200/600], Loss: 0.0164
Epoch [4/5], Step [300/600], Loss: 0.1066
Epoch [4/5], Step [400/600], Loss: 0.0250
Epoch [4/5], Step [500/600], Loss: 0.0462
Epoch [4/5], Step [600/600], Loss:

### When i used both Nadam optimiser and He wiight initialisatio, the model had the accuracy of 99.06% but it can be observed that train loss is comparatively less than base model.

Hence we should try different weight initlisation and optimiser to find a hyperparameter that works with problem at hand